# Scraper des Discours

This notebook is foreked from:
https://github.com/Araxide/french_speech_scraper/blob/master/discours_scraper.py
I have updated the calls selenium's find_elements_by* functions.
Many thanks!

In [1]:
#mettre à jour Chromium ou Chrome version=Version 114.0.5735.90 (Official Build) Arch Linux (64-bit)
#!pip install webdriver-manager

In [2]:
#!pip install selenium # donc on utilise selenium 4!

In [3]:
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [4]:
from selenium.webdriver.chrome.service import Service as ChromeService
class DiscoursScraper():
  
    def __init__(self):
        #self.driver = webdriver.Chrome(ChromeDriverManager().install())
        service = Service("/usr/bin/chromium")
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        # à changer si tu as des critères de recherche
        #self.main_page_url = 'https://www.vie-publique.fr/discours/recherche?search_api_fulltext_discours=&sort_by=field_date_prononciation_discour&field_intervenant_title=&field_intervenant_qualite=&field_date_prononciation_discour_interval%5Bmin%5D=&field_date_prononciation_discour_interval%5Bmax%5D=&field_type_emetteur=&field_type_de_document='
        #self.child_page_url = 'https://www.vie-publique.fr/discours/recherche?search_api_fulltext_discours=&sort_by=field_date_prononciation_discour&field_intervenant_title=&field_intervenant_qualite=&field_date_prononciation_discour_interval%5Bmin%5D=&field_date_prononciation_discour_interval%5Bmax%5D=&field_type_emetteur=&field_type_de_document=&page=1'
    
        #ça marche pour la plupart de discours 2019-2023
        '''
        self.main_page_url = 'https://www.vie-publique.fr/discours/recherche?search_api_fulltext_discours=&sort_by=field_date_prononciation_discour&field_intervenant_title=&field_intervenant_qualite=&field_date_prononciation_discour_interval%5Bmin%5D=&field_date_prononciation_discour_interval%5Bmax%5D=&field_type_de_document%5B%5D=&field_type_emetteur%5B%5D=9340&form_build_id=form-5OkWQR9wk6heLS7UrXWRSOMkhi_8G94YkOvZzq_l8fg&form_id=views_exposed_form'
        self.child_page_url = 'https://www.vie-publique.fr/discours/recherche?search_api_fulltext_discours=&sort_by=field_date_prononciation_discour&field_intervenant_title=&field_intervenant_qualite=&field_date_prononciation_discour_interval%5Bmin%5D=&field_date_prononciation_discour_interval%5Bmax%5D=&field_type_de_document%5B0%5D=&field_type_emetteur%5B0%5D=9340&form_build_id=form-5OkWQR9wk6heLS7UrXWRSOMkhi_8G94YkOvZzq_l8fg&form_id=views_exposed_form&page='
        '''

        #mieux pour les discours  avant 2019, on ne rate rien mais on doit nettoyer les html après
        self.main_page_url = 'https://www.vie-publique.fr/discours/recherche?search_api_fulltext_discours=&sort_by=field_date_prononciation_discour&field_intervenant_title=&field_intervenant_qualite=&field_date_prononciation_discour_interval%5Bmin%5D=1974-05-27&field_date_prononciation_discour_interval%5Bmax%5D=2018-12-31&field_type_emetteur%5B0%5D=9340&field_type_de_document%5B0%5D='
        self.child_page_url = 'https://www.vie-publique.fr/discours/recherche?search_api_fulltext_discours=&sort_by=field_date_prononciation_discour&field_intervenant_title=&field_intervenant_qualite=&field_date_prononciation_discour_interval%5Bmin%5D=1974-05-27&field_date_prononciation_discour_interval%5Bmax%5D=2018-12-31&field_type_emetteur%5B0%5D=9340&field_type_de_document%5B0%5D=&page='
        
        # à changer selon vos besoins
        self.pages_begin = 816
        self.pages_end = 823 #937 #100 

    def scrap_speeches_urls(self, page_url):
        self.driver.get(page_url)
        sleep(3) # wait until all elements have loaded
        articles = self.driver.find_elements(By.CSS_SELECTOR, "h3.fr-card__title > a")
        articles = [article.get_attribute('href') for article in articles]
        return articles

    def record_speeches_of_page(self, writer, speeches_urls):
        for speech_url in speeches_urls:
            self.driver.get(speech_url)
            try:                
                sleep(1) # wait until all elements have loaded
                discours = self.driver.find_elements(By.CSS_SELECTOR, "div.vp-discours-content > div > p")
                discoursText = ""            
                for d in discours:
                    discoursText += self.remove_line_breaks(d)
                # discours title  
                title = self.driver.find_element(By.CSS_SELECTOR, "h1")
                # discours date
                date_discours = self.driver.find_element(By.CSS_SELECTOR, "time")
                # append row on csv file
                writer.writerow({'titre': title.text, 'date': date_discours.text, 'discours': discoursText})
            except (NoSuchElementException):
                pass #pour l'instant on l'ignore
            
    def record_speeches_of_page_before_2019(self, writer, speeches_urls):
        for speech_url in speeches_urls:
            self.driver.get(speech_url)
            try:                
                sleep(2) # wait until all elements have loaded
                discoursText = self.driver.find_element(By.CSS_SELECTOR, "div.vp-discours-content > div").get_attribute('innerHTML')
                # discours title  
                title = self.driver.find_element(By.CSS_SELECTOR, "h1")
                # discours date
                date_discours = self.driver.find_element(By.CSS_SELECTOR, "time")
                # append row on csv file
                writer.writerow({'titre': title.text, 'date': date_discours.text, 'discours': discoursText})
            except (NoSuchElementException):
                pass #pour l'instant on l'ignore
    
    def remove_line_breaks(self, element):
        element = element.text.replace('\n', ' ').replace('\r', '') # remove line breaks in text
        return str(element)
  
    def main(self):
        # create csv
        with open('scaped_discours-Président_2018-1974.csv', 'a', encoding='utf-8', newline='') as csvfile:
            fieldnames = ['titre', 'date', 'discours']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            # scrap first page -- à enlever si on ne commence pas par le page 0
            speeches_urls_first_page = self.scrap_speeches_urls(self.main_page_url) #2019-2023
            #self.record_speeches_of_page(writer, speeches_urls_first_page)   
            self.record_speeches_of_page_before_2019(writer, speeches_urls_first_page)
            
            # get speeches urls for each next page until reach pages_total
            for i in range(self.pages_begin, self.pages_end):
                speeches_urls = self.scrap_speeches_urls(self.child_page_url+str(i))
                # scrap speech text
                #self.record_speeches_of_page(writer, speeches_urls) #2019-2023
                self.record_speeches_of_page_before_2019(writer, speeches_urls)

        # end webdriver session    
        self.driver.quit()

In [38]:
DiscoursScraper().main()

[WDM] - Downloading: 100%|██████████| 7.06M/7.06M [00:00<00:00, 52.7MB/s]


# Read the dataset

In [5]:
import pandas as pd

In [ ]:
df_1974_2018 = pd.read_csv('scaped_discours-Président_2018-1974.csv')

In [42]:
df_2019_2023 = pd.read_csv('scraped_discours-Président_2023-2019.csv')

In [43]:
df_2019_2023 = df_2019_2023.drop_duplicates(subset=['discours'])
df_2019_2023.describe()

,titre,date,discours
count,317,317,316
unique,317,253,316
top,"Déclaration de M. Emmanuel Macron, président d...",30 janvier 2020,"Merci beaucoup Monsieur le Président, je suis ..."
freq,1,4,1


In [40]:
df_2019_2023['discours'].isnull().sum()

1

In [46]:
df_2019_2023.dropna(subset=['discours'], inplace=True)
df_2019_2023['discours'].isnull().sum()

0

In [47]:
df_2019_2023.to_csv('cleaned_discours-Président_2023-2019.csv')

In [26]:
import locale
from dateutil.parser import parse as parse_dt
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')   ## first I set locale

'fr_FR.UTF-8'

In [69]:
df_1974_2018 = pd.read_csv('cleaned_discours-Président_2018-1974.csv', index_col=0)

In [61]:
df_2019_2023 = pd.read_csv('cleaned_discours-Président_2023-2019.csv', index_col=0)

In [62]:
df_1974_2018.tail()

,titre,date,discours
69,ALLOCUTION RADIOTELEVISEE PRONONCEE PAR M. VAL...,31 décembre 1974,"BONNE ANNEE POUR CHACUNE DE VOUS, BONNE ANNEE ..."
70,ALLOCUTION DE M. LE PRESIDENT DE LA REPUBLIQUE...,23 décembre 1974,"<p>MONSIEUR L'AMBASSADEUR, =POLITIQUE EXTERIEU..."
71,ALLOCUTION DE M. LE PRESIDENT DE LA REPUBLIQUE...,23 décembre 1974,"<p>MONSIEUR L'AMBASSADEUR, =POLITIQUE EXTERIEU..."
72,ALLOCUTION DE M. LE PRESIDENT DE LA REPUBLIQUE...,23 décembre 1974,"<p>MONSIEUR L'AMBASSADEUR, =POLITIQUE EXTERIEU..."
73,ALLOCUTION DE M. VALERY GISCARD D'ESTAING A L'...,15 décembre 1974,"<p>MONSIEUR LE PRESIDENT, MESDAMES, MESSIEURS,..."


# Remove html tags

In [70]:
df_1974_2018['discours'] = df_1974_2018['discours'].str.replace("<br>",'\n')
df_1974_2018['discours'] = df_1974_2018['discours'].str.replace(r'<[^<>]*>', '', regex=True)

# Parse French date

In [64]:
#!pip install dateparser
import dateparser
dateparser.parse('31 décembre 1974').date()

datetime.date(1974, 12, 31)

In [65]:
df_2019_2023['date'] = df_2019_2023['date'].map(lambda dt: dateparser.parse(dt).date())
df_2019_2023.to_csv('cleaned_discours-Président_2023-2019_without_HTML.csv',index=False)
df_2019_2023.tail()

,titre,date,discours
439,"Déclaration de M. Emmanuel Macron, Président d...",2018-01-19,"Mesdames les ministres, Mesdames et messieurs ..."
452,"Déclaration de M. Emmanuel Macron, Président d...",2018-01-04,"Monsieur le Premier Ministre, Mesdames, Messie..."
478,"Déclaration de M. Emmanuel Macron, Président d...",2017-12-06,"Bonjour Messieurs Dames, Je m'exprimerai plus ..."
484,"Déclaration de M. Emmanuel Macron, Président d...",2017-11-25,"Monsieur le Premier ministre, Mesdames, Messie..."
499,"Déclaration de M. Emmanuel Macron, Président d...",2017-11-08,"Mesdames, messieurs les Ministres, Monsieur l'..."


In [71]:
df_1974_2018['date'] = df_1974_2018['date'].map(lambda dt: dateparser.parse(dt).date())
df_1974_2018.to_csv('cleaned_discours-Président_2018-1974_without_HTML.csv',index=False)

In [72]:
# now you can read the csv with 
df_1974_2018_clean = pd.read_csv('cleaned_discours-Président_2018-1974_without_HTML.csv', parse_dates=['date'])
#df_2019_2023_clean = pd.read_csv('cleaned_discours-Président_2023-2019_without_HTML.csv', parse_dates=['date'])

In [73]:
df_1974_2018_clean.tail()

,titre,date,discours
9146,ALLOCUTION RADIOTELEVISEE PRONONCEE PAR M. VAL...,1974-12-31,"BONNE ANNEE POUR CHACUNE DE VOUS, BONNE ANNEE ..."
9147,ALLOCUTION DE M. LE PRESIDENT DE LA REPUBLIQUE...,1974-12-23,"MONSIEUR L'AMBASSADEUR, =POLITIQUE EXTERIEURE=..."
9148,ALLOCUTION DE M. LE PRESIDENT DE LA REPUBLIQUE...,1974-12-23,"MONSIEUR L'AMBASSADEUR, =POLITIQUE EXTERIEURE=..."
9149,ALLOCUTION DE M. LE PRESIDENT DE LA REPUBLIQUE...,1974-12-23,"MONSIEUR L'AMBASSADEUR, =POLITIQUE EXTERIEURE=..."
9150,ALLOCUTION DE M. VALERY GISCARD D'ESTAING A L'...,1974-12-15,"MONSIEUR LE PRESIDENT, MESDAMES, MESSIEURS, =P..."
